# 09. Milestone Project 2: SkimLit 📄🔥

Önceki not defterinde (TensorFlow'da NLP temelleri), bazı temel doğal dil işleme kavramlarını inceledik. Bunlardan başlıcaları tokenzasyon (sözcükleri sayılara dönüştürmek) ve katıştırmalar oluşturmaktır (sözcüklerin sayısal bir temsilini oluşturmak). Bu projede öğrendiklerimizi hayata geçireceğiz. Daha spesifik olarak, 2017 tarihli PubMed 200k RCT: a Dataset for Sequenctial Cümle Sınıflandırması Tıbbi Özetlerde arkasındaki derin öğrenme modelini kopyalayacağız. Makale yayınlandığında, ~200.000 etiketli Randomize Kontrollü Deneme (RCT) özetinden oluşan PubMed 200k RCT adlı yeni bir veri seti sundu. Veri setinin amacı, NLP modellerinin sıralı sırada görünen cümleleri sınıflandırma yeteneğini keşfetmekti. Başka bir deyişle, bir RCT'nin özeti verildiğinde, özette her cümlenin rolü nedir?

## Confirm access to a GPU

In [1]:
# Check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-bbd2ea82-3388-699b-0a04-24b58e61280e)


## Get data

In [2]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 33 (delta 5), reused 5 (delta 5), pack-reused 25
Receiving objects: 100% (33/33), 177.08 MiB | 5.71 MiB/s, done.
Resolving deltas: 100% (12/12), done.
PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


In [3]:
# Check what files are in the PubMed_20K dataset
!ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign

dev.txt  test.txt  train.txt


In [4]:
# Start by using the 20k dataset
data_dir = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

In [5]:
# Check all of the filenames in the target directory
import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 'pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt',
 'pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt']

## Preprocess data

Tamam, şimdi bazı metin verilerini indirdik, sence onu modellemeye hazır mıyız? Bekle... Verileri indirdik ama henüz ona bakmadık bile. Herhangi bir yeni veri kümesine aşina olmanın sloganı nedir? Size bir ipucu vereyim, kelime "v" ile başlıyor ve üç kez söylüyoruz. vibe mı, vibe mı, vibe mı? Bir nevi... kesinlikle verilerimizin titreşimini hissetmeliyiz. Değerler, değerler, değerler? Yine, pek çok değer görmek istiyoruz ama tam olarak aradığımız şeyi değil. Görselleştirmek, görselleştirmek, görselleştirmek? Boom! Bu kadar. Derin öğrenme modellerimiz için verilerimizi nasıl hazırlamamız gerektiğini öğrenmek ve anlamak için onu görselleştirmemiz gerekiyor. Verilerimiz metin dosyaları biçiminde olduğundan, bir hedef dosyadaki satırların her birini okumak için bazı kodlar yazalım.


In [6]:
# Create function to read the lines of a document
def get_lines(filename):
  """
  Reads filename (a text file) and returns the lines of text as a list.

  Args:
      filename: a string containing the target filepath to read.

  Returns:
      A list of strings with one string per line from the target filename.
      For example:
      ["this is the first line of filename",
       "this is the second line of filename",
       "..."]
  """
  with open(filename, "r") as f:
    return f.readlines()

In [7]:
train_lines = get_lines(data_dir+"train.txt")
train_lines[:20] # the whole first example of an abstract + a little more of the next one

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

Let's think about how we want our data to look...

How I think our data would be best represented ...

[{'line_number':0,
  'target':'BACKGROUND',
  'text':"Emotional eating is associated with overeating and the development of obesity .\n'",
  'total_lines':11}]